# Section1: Experiment Search Console


In [1]:
import os,sys,inspect
path=os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.append(os.path.abspath(path+"/../../"))
path=os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.append(os.path.abspath(path+"/../"))
path=os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.append(os.path.abspath(path+"/../components/"))
path=os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.append(os.path.abspath(path+"/../dashboards/"))
sys.path.append(os.path.abspath(path+"/../../signals/"))

import SystemConsole
from SystemConsole import *
import  BokehComponents
from BokehComponents import *

from conf.conf import conf

from signals.Process import SignalProcess

import pymongo
from pymongo import MongoClient

from bson import ObjectId

Loading BokehJS ...

3.1.1
3.1.1


In [2]:
# Define our buttions
class EmailButton(BokehButton):
    def handle_click(self,event):
        sp = SignalProcess()
        sp.proc_init()
        sp.step_mainbody(force_email=True)

class KillButton(BokehButton):
    def handle_click(self,event):
        for t in self._settings['tables_to_kill']:
            #print('calling')
            t.removeSelected()
            #print('called')


class ShowButton(BokehButton):
    def handle_click(self,event):
        rows = []
        for targetTable in self._settings['analytics_targets']:
            selected = targetTable.getSelected()
            if len(selected) > 0:
                selected = selected[0]
                row = {}
                for col in targetTable.data.keys(): 
                    row[col]=targetTable.data[col][selected]
                rows.append(row)
        ## Load the experiment manager, and get analytics
        
        ad = self._settings['analytics_div']
        ad.text = str(rows)

In [3]:
from datetime import date,datetime,timedelta
from random import randint
from bokeh.io import output_file, show
from bokeh.layouts import widgetbox, Spacer
from bokeh.models import ColumnDataSource,CustomJS,Div
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn, Tabs, Panel
from bokeh.models.widgets import Button, RadioButtonGroup, Select, Slider
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import layout
from bokeh.events import ButtonClick
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
import pandas as pd
import pandas_datareader as pdr
from bokeh.plotting import figure,show,output_file

from BokehComponents import *

output_notebook()
output_file("data_table.html")
class AdviceConsole:
    def __init__(self):
        self.div = Div(width=300)
        table_list = [
            {'buffered_query_interface':UserQuery(),'id_field':'_id'},
            {'buffered_query_interface':PoolQuery(),'id_field':'_id'},
            {'buffered_query_interface':SignalQuery(),'id_field':'_id'},
            {'buffered_query_interface':BracketQuery(),'id_field':'_id'},
            {'buffered_query_interface':AutotradeQuery(),'id_field':'_id'},
            {'buffered_query_interface':EmailQuery(),'id_field':'_id'},
            {'buffered_query_interface':SystemConsole.AlgorithmQuery(),'id_field':'aid'},
            {'buffered_query_interface':SystemConsole.ExperimentQuery(),'id_field':'eid'}]
            
        self.tables = []
        for table in table_list:
            self.tables.append(QueryTableComponent({'buffered_query_interface':table['buffered_query_interface'],
                                                    'id_field':table['id_field']}))
        
        self.analytics_div = Div(text="""Analytics HTML""", width=200, height=100)
        self.console = Div(text="""System Out console:""", width=200, height=100)
        
        self.b_kill = KillButton({'tables_to_kill':self.tables,'label':'kill'})
        self.b_show = ShowButton({'label':'show','analytics_targets':self.tables,'analytics_div':self.analytics_div})
        self.b_restart = ShowButton({'label':'restart'})
        self.b_email = EmailButton({'label':'send emails now'})
        #self.time_graphic = TimeseriesGraphic()
        #self.periodicCallbacks = [self.time_graphic.callback_refresh_analytics]
        self.periodicCallbacks = []
        
        self.s_adviceSelect = BokehSelect({'title':"Advice On:", 'value':"Auto", 'options':["Yes", "No", "Auto"]})
        self.s_tradeSelect = BokehSelect({'title':"Trade:", 'value':"Auto", 'options':["Yes", "No", "Auto"]})
        self.s_autoRestart = BokehSelect({'title':"Autostart:", 'value':"No", 'options':["Yes", "No"]})
        
    def getPeriodicCallbacks(self):
        return self.periodicCallbacks
        
    def getControlPanel(self):
        
        bk = self.b_kill.getBokehComponent()
        br = self.b_restart.getBokehComponent()
        bs = self.b_show.getBokehComponent()
        be = self.b_email.getBokehComponent()
        adviceSelect = self.s_adviceSelect.getBokehComponent()
        tradeSelect  = self.s_tradeSelect.getBokehComponent()
        autoRestart  = self.s_autoRestart.getBokehComponent()
        
        c_tab1 = Panel(child=layout([bk,bs,br,be], sizing_mode='fixed'),title="Commands")
        c_tab2 = Panel(child=layout([adviceSelect,tradeSelect,autoRestart], sizing_mode='fixed'),title="Signals")
        c_tabs = Tabs(tabs=[ c_tab1,c_tab2], width=300)
        
        return widgetbox(c_tabs)


    def getConsole(self):

        #(Compute Nodes, Running Experiments, Queue and History, Orders & Advice )
        #financePlot = self.time_graphic.getBokehComponent()

        j_tab1 = Panel(child=layout([self.analytics_div], sizing_mode='fixed'),title="Analytics")
        j_tab2 = Panel(child=layout([Div(height=500,width=500)], sizing_mode='fixed'),title="System Out")
        j_tab3 = Panel(child=layout([Div(height=500,width=500)], sizing_mode='fixed'),title="Log Data")
        j_tab4 = Panel(child=layout([Div(height=500,width=500)], sizing_mode='fixed'),title="Job Code")
        j_tabs = Tabs(tabs=[ j_tab1, j_tab2, j_tab3, j_tab4 ], width=700)
        
        display_sections = []
        for t in self.tables:
            display_sections.append(t.getBokehComponent())
            
        display_sections.append([self.getControlPanel(),j_tabs])
            
        l2 = layout(display_sections,sizing_mode='fixed') 
        

        tab2 = Panel(child=l2,title="User Analytics")
        tab3 = Panel(child=Div(height=300,width=600),title="Orders & Integration")
        tabs = Tabs(tabs=[tab2, tab3 ])
        return tabs
    


Loading BokehJS ...

In [4]:

# Create the Document Application
global fd,ee
def modify_doc(doc):
    
    # Create the main plot
    def create_figure():
        global fd,ee        
        ee = AdviceConsole()
        return ee.getConsole()
    
    # Update the plot
    def update(attr, old, new):
        print('test')
    
    doc.add_root(create_figure())
    pc_list = ee.getPeriodicCallbacks()
    for callback in pc_list:
        doc.add_periodic_callback(callback, 500)

def showConsole(    notebook_url="54.184.199.101:9999"):
    app = Application( FunctionHandler(modify_doc))
    doc = app.create_document()
    show(app, notebook_url=notebook_url)

In [5]:
from conf.conf import conf
showConsole(conf.get("this_notebook_server","54.184.199.101:9999"))

{'_id': ObjectId('5c0ee9117596c332a95f2fdc'), 'sec': 'APD', 'date': datetime.datetime(2018, 12, 10, 22, 26, 33, 739000), 'sell': datetime.datetime(2018, 12, 10, 22, 46, 33, 739000), 'high': 165.0495, 'low': 149.3305, 'conf': 1, 'pl': 1.0035348832177047, 'used': False, 'week': 50}
{'_id': ObjectId('5c0ee9117596c332a95f2fdb'), 'sec': 'BDX', 'date': datetime.datetime(2018, 12, 10, 22, 26, 33, 739000), 'sell': datetime.datetime(2018, 12, 10, 22, 46, 33, 739000), 'high': 248.32500000000002, 'low': 224.67499999999998, 'conf': 1, 'pl': 1.0405831751043322, 'used': False, 'week': 50}
{'_id': ObjectId('5c0ee9117596c332a95f2fd6'), 'sec': 'EMN', 'date': datetime.datetime(2018, 12, 10, 22, 26, 33, 739000), 'sell': datetime.datetime(2018, 12, 10, 22, 46, 33, 739000), 'high': 76.3875, 'low': 69.1125, 'conf': 1, 'pl': 1.0014503165975825, 'used': False, 'week': 50}
{'_id': ObjectId('5c0ee9117596c332a95f2fd2'), 'sec': 'BBBY', 'date': datetime.datetime(2018, 12, 10, 22, 26, 33, 739000), 'sell': datetime.

ERROR:bokeh.server.protocol_handler:error handling message Message 'EVENT' (revision 1): FileNotFoundError(2, 'No such file or directory')


[[], [], [], []]
['APD', 'BDX', 'EMN', 'BBBY']
100
[]
ValueError('Cannot convert -1 to a date.  This often happens if non-datetime values are passed to an axis that expects datetime objects.',)
email send
christopher.hemming@alumni.utoronto.ca
<div class="wrapper" style="width: 80%; border: solid 1px #ccc; padding: 3rem; margin: 2rem auto; background-color: white;"> <div class = "header_section" style="text-align: center;"><img src="cid:paxlogo.jpg" width="200"><br /> <h2 style="font-size: 2rem;">Pax Cultura Studios</h2> </div> <div class = "intro_section" style="text-align: center; font-size: 1rem; margin: 3rem 0; border-top: solid 1px #ccc; border-bottom: solid 1px #ccc; padding: 2rem 0;"> <strong>Hello gyorgi_test!</strong> <p>Thank you for subscribing to Pax Finance platform!<br />Please find a list of recommended trades created specifically for you in the table below:</p> </div> <div class = "recomend_section"><table style="width: -webkit-fill-available;"><tr><td>Security</td><td>

In [ ]:
### Job Management
# [ ] select: Queue, Running, Halted, Finihsed
# [ ] search: MongoDB settings query 
# [ ] Parent / Child Filtering
# [ ] Parent = All Jobs present
# [ ] Parent = No Parent
# [X] running experiments in tab 2
# [X] get kills working
# [X] get show button working
# [ ] get sub experiments listing based on experiment
# [ ] autostart property on Experiment
# [ ] See job settings

### Orders & Advice System
# [ ] See linked in traction
# [ ] For each Experiment (running / historical):
# [ ] Sent Brackets + P/L
# [ ] Pending Brackets + P/L
# [ ] Raw P/L
# [ ] Advice (On / Off)
# [ ] Trading (On / Off)



In [ ]:
ee.kill()

In [ ]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook

# Create a blank figure with labels
p = figure(plot_width = 600, plot_height = 600, 
           title = 'Example Glyphs',
           x_axis_label = 'X', y_axis_label = 'Y')

# Example data
squares_x = [1, 3, 4, 5, 8]
squares_y = [8, 7, 3, 1, 10]
circles_x = [9, 12, 4, 3, 15]
circles_y = [8, 4, 11, 6, 10]

# Add squares glyph
p.square(squares_x, squares_y, size = 12, color = 'navy', alpha = 0.6)
# Add circle glyph
p.circle(circles_x, circles_y, size = 12, color = 'red')

# Set to output the plot in the notebook
output_notebook()
# Show the plot
show(p)

In [ ]:
from pymongo import MongoClient
mf = mongoInfo
display(mf)
client=MongoClient(host=mf['mongo_server'],port=mf['port'],username=mf['username'],password=mf['password'],authSource=mf['authSource'])
display(client)

In [ ]:
display(client.list_database_names())

In [ ]:
str_db = client.list_database_names()[0]
display(str_db)

In [ ]:
display(client[str_db].collection_names())


In [ ]:
import pprint
cur = client['fleetRover']['yahoo'].find().limit(500)
data = []
for f in cur:
    data.append(f)
df = pd.DataFrame(data)
display(df)

In [ ]:
d = df[['High','Low']]
display(d)

In [ ]:
# Read the data from a csv into a dataframe
# Summary stats for the column of interest
d['High'].describe()


In [ ]:
#Develop a Histogram
import numpy as np
arr_hist, edges = np.histogram(d['High'], 
                               bins = int(180/5), 
                               range = [-60, 120])
# Put the information in a dataframe
delays = pd.DataFrame({'arr_delay': arr_hist, 
                       'left': edges[:-1], 
                       'right': edges[1:]})

# Create the blank plot
p = figure(plot_height = 600, plot_width = 600, 
           title = 'Histogram',
          x_axis_label = 'Amount]', 
           y_axis_label = 'Number')

# Add a quad glyph
p.quad(bottom=0, top=d['Low'], 
       left=delays['left'], right=delays['right'], 
       fill_color='red', line_color='black')

# Show the plot
show(p)